In [ ]:
from __future__ import annotations
import datetime as dt
from typing import NamedTuple
from collections.abc import Iterable
from random import random
import sys
from contextlib import contextmanager

from tuplesaver.engine import Engine
from tuplesaver.model import Row, TableRow
from tuplesaver.adaptconvert import AdaptConvertTypeAlreadyRegistered, included_adapt_convert_types

In [ ]:
from collections.abc import Iterator
import apsw
import re
import apsw.ext

class ListType(TableRow):
    codename: str


class List(TableRow):
    name: str  # unique
    listtype: ListType

class ListSection(TableRow):
    title: str
    list: List


class ListItem(TableRow):
    txt: str
    listsection: ListSection


@contextmanager
def engine() -> Iterator[Engine]:
    conn = apsw.Connection("xxx.db")
    with apsw.ext.Trace(sys.stdout, conn):
        conn.pragma("journal_mode", "WAL")

        with conn:
            e = Engine(conn)
            e.adapt_convert_registry.register_included_adaptconverters(included_adapt_convert_types)

            e.ensure_table_created(ListType)
            e.ensure_table_created(List)
            e.ensure_table_created(ListSection)
            e.ensure_table_created(ListItem)

        with e.connection:
            yield e

with engine() as e:
    shopping = e.save(ListType("shopping"))
    groceries = e.save(List("Grocery", shopping))

# Example of verbose loading because of lack of backrefs/lazy loading
use this to drive the design

In [ ]:
# populate the database with some data
with engine() as e:
    section_items = {
        'produce': [
            'Other fruit',
            'Berries',
            'Limes',
            'oranges, not the very large ones, but not the small ones either. Also they should be extra sweet, and heavy (and therefore JUICY 💦 yum!)',
        ],
        'deli': ['sliced cheese', 'Lunch meat', 'Goat cheese'],
        'meat': ['chicken'],
        'dairy': ['Milk', 'Eggs', 'Sour cream', 'Cheddar block'],
        'frozen': ['Frozen Veggies'],
        'drinks': ['sparkling water', 'Pop'],
        'snacks': ['peanuts', 'Triscut', 'Goldfish'],
        'ingredient': ['tortillas', 'Peanut butter', 'Prescription'],
        'etc': ['cat food', 'Tide'],
    }

    for section, items in section_items.items():
        section = e.save(ListSection(section, groceries))
        for item in items:
            e.save(ListItem(item, section))


In [ ]:
from typing import cast
from tuplesaver.sql import select

@select(ListItem)
def listitems_by_listname(list_name: str):
    return f"WHERE {ListItem.listsection.list.name} = {list_name}"


list_name = "Grocery"

@select(List)
def list_by_name(list_name: str):
    return f"WHERE {List.name} = {list_name}"

with engine() as e:
    mylist = e.query(*list_by_name(list_name)).fetchone()

# Alternative: using find_by
with engine() as e:
    mylist = e.find_by(List, name=list_name)

if mylist is None:
    print(f"List {list_name} not found")

with engine() as e:
    listitems = e.query(*listitems_by_listname(list_name)).fetchall()

if len(listitems) == 0:
    print(f"No items found for list {list_name}")

section_items = {}
for item in listitems:
    try:
        print(item)
        section_items.setdefault(item.listsection.title, []).append(item)
        print(f"Added item {item.txt} to section {item.listsection.title}")
    except AttributeError:
        print(f"Item {item} has no listsection assigned")

print(f"Rendering template for list {mylist}")